<br>
<a href="https://www.nvidia.cn/training/">
    <div style="width: 55%; background-color: white; margin-top: 50px;">
    <img src="https://dli-lms.s3.amazonaws.com/assets/general/nvidia-logo.png"
         width="400"
         height="186"
         style="margin: 0px -25px -5px; width: 300px"/>
</a>
<h1 style="line-height: 1.4;"><font color="#76b900"><b>使用大语言模型（LLM）构建 AI 智能体</h1>
<h2><b>练习 3:</b> 在 LangGraph 中实现个性化系统</h2>
<br>

在前面的部分中，您探索了如何使用 LangGraph 来协同处理节点和边，以实现基本的智能工作流。现在，我们来回顾一下第一节中关于老师、学生和家长的个性化智能体问题。我们现在有了结构化的输出和 LangGraph 经验，所以也许可以利用这个新抽象来协同系统？您将看到如何设置每个个性化智能体的数据，创建统一的提示词格式，生成结构化 JSON 响应，并将这些智能体链接在一起形成状态图。

### **学习目标:**
**在这个 notebook 中，我们将:**

- 熟悉 LangGraph 及其状态管理抽象。
- 基于 LLM 的方向实施合理的路由（与我们在 Notebook 2t 中失败的 ReAct 尝试形成对比）。

In [ ]:
from langchain_nvidia import ChatNVIDIA

llm = ChatNVIDIA(model="meta/llama-3.1-8b-instruct", base_url="http://nim-llm:8000/v1")

<hr><br>

### **第一部分:** 引入个性化智能体

您可能还记得我们在基础 Python 中定制的系统和在 CrewAI 中简化的系统，下面就把这些规范整合到一起，创建几个个性化智能体:

In [ ]:
teacher_args = dict(
    role="John Doe (Teacher)",
    backstory=(
        "You are a computer science teacher in high school holding office hours, and you have a meeting."
        " This is the middle of the semester, and various students have various discussion topics across your classes."
        " You are having a meeting right now. Please engage with the students and help their parent."
    ), 
    directive="You are having a meeting right now. Please engage with the other speakers and help them out with their concerns.",
)

student1_args = dict(
    role="Jensen (Student)",
    backstory="You are taking Dr. Doe's intro to algorithms course and are struggling with some of the homework problems.", 
    directive="Meet with your teacher to help you understand class material. Respond and ask directed questions, contributing to discussion.",
)

student2_args = dict(
    role="Albert (Student)",
    backstory="You are taking Dr. Doe's intro to algorithms course and are struggling with some of the homework problems.", 
    directive="Meet with your teacher to help you understand class material. Respond and ask directed questions, contributing to discussion.",
)

parent_args = dict(
    role="Sally (Parent)",
    backstory="You are here with your kids, who are students in the teacher's class.", 
    directive="Meet with your kids and the teacher to help support the students and see what you can do better.",
)

agent_unique_spec_dict = {args.get("role"):args for args in [teacher_args, student1_args, student2_args, parent_args]}

<br>

现在，我们来构建一个足够灵活的 `ChatPromptTemplate`，可以应用于每个个性化智能体。您将看到以下占位符：
- `{role}`、`{backstory}` 和 `{directive}`，来自智能体规范。
- 一个最终的 `schema_hint` 空间，用于帮助我们路由系统。
- 包含迄今为止对话消息的消息占位符。



In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from course_utils import SCHEMA_HINT  ## <- Convenience method to get schema hint template

## Define the structured prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", (
        "You are {role}. {backstory}"
        "\nThe following people are in the room: {role_options}."
        " {directive}\n" f"{SCHEMA_HINT}"
    )),
    ("placeholder", "{messages}"),
])

<hr><br>

### **步骤 2:** 定义响应模式

利用与 Notebook 2 相似的逻辑，我们可以为系统赋予结构化输出，不仅能获取自然语言响应，还可以生成路径变量，随后用于路由对话。

基于当前状态的合法路径指导解码有点难以管理，但可以通过调整发送到 LLM 入口的模式来控制。以下定义了一个便捷方法 `get_finite_schema`。

In [ ]:
from pydantic import BaseModel, Field
from typing import Any, Dict, List, Literal

## Definition of Desired Schema
class AgentResponse(BaseModel):
    """
    Defines the structured response of an agent in the conversation.
    Ensures that each agent response includes the speaker's identity,
    a list of response messages, and a defined routing option.
    """
    speaker: Literal["option1", "option2"] = Field(description="Who are you responding as?")
    response: List[str] = Field(description="Response to contribute to the conversation")
    route: Literal["option1", "option2"] = Field(description="A choice of the next person")

    @classmethod
    def get_default(cls):
        return cls(speaker="option1", response=[], route="option1")
    
    @classmethod
    def get_finite_schema(cls, key_options: Dict[str, List[str]]) -> Dict[str, Any]:
        """
        Dynamically modifies the schema to adjust the possible routing options.
        This is useful for ensuring the model respects dynamic conversation flows.
        """
        schema = cls.model_json_schema()
        for key, options in key_options.items():
            if "enum" in schema["properties"].get(key, {}):
                schema["properties"][key]["enum"] = options
            if "items" in schema["properties"].get(key, {}):
                schema["properties"][key]["items"] = {'enum': options, 'type': 'string'}
        return schema

role_options = list(agent_unique_spec_dict.keys()) + ["End"]
schema_hint = AgentResponse.get_finite_schema({"speaker": role_options[:1], "route": role_options})
schema_hint

<br>

就这样，现在有了填充提示词模板所需的本地和全局规范。这将作为我们构建专用**智能体**类的参数。

In [ ]:
## Shared parameters across agents
shared_args = dict(
    llm=llm, 
    schema=AgentResponse.get_default(), 
    schema_hint=schema_hint, 
    prompt=prompt, 
    routes=role_options, 
    roles=role_options
)

## Initialize agent specifications with shared parameters
agent_spec_dict = {
    role: {**unique_specs, **shared_args} 
    for role, unique_specs in agent_unique_spec_dict.items()
}

<hr><br>

### **步骤 3:** 定义智能体类

为了简化最终的调度图的复杂性，我们可以实现一些有状态的智能体，就像在 CrewAI 示例中那样。 

- 为了遵循引导智能体理论的方法，通往 LLM 的接口被称为 `_convert_to_local` 和 `_convert_to_global`。如果您查看它们，会发现它们看起来非常熟悉。
- 您会注意到在 `_get_llm` 中，我们用可能希望 LLM 选择的类对 `get_finite_method` 进行了参数化（或没有选择的情况）。请注意，这并不是 LangChain/LangGraph 官方支持的方法，只是为了简化代码库。
- 上次没有强调这一点，但您会注意到我们使用 `.invoke` 调用了 llm。如果调用类的时候开始流式输出，这可就奇怪了...

<details><summary><b>提示词</b></summary>

确保利用好您的消息缓冲区。也许第一步是将生成的消息添加到缓冲区？接着，我们只需根据响应路由到适当的智能体...

</details>


<details><summary><b>参考答案</b></summary>

```python
## Start with the first agent
teacher_agent = Agent(**list(agent_spec_dict.values())[0])
response = teacher_agent(messages=messages)[0]
print(response)
messages.append((response.get("speaker"), response.get("response")))

## TODO: Route to the next agent based on response
next_agent = Agent(**agent_spec_dict.get(response.get("route"), {}))
response = next_agent(messages=messages)[0]
print(response)
messages.append((response.get("speaker"), response.get("response")))

## TODO: Continue the conversation
next_agent = Agent(**agent_spec_dict.get(response.get("route"), {}))
response = next_agent(messages=messages)[0]
print(response)
```

</details>

<hr><br>

### **第四部分:** 整合所有内容

现在有了所有这些组件，可以将它们整合在一起，构建适合我们用例的智能体系统。和之前一样，可以仅用一个智能体抽象来完成这一切，但每个智能体可以有自己的类实例。作为一个练习，看看您能否在不查看参考答案的情况下构建智能体类！

In [ ]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.constants import START, END
from langgraph.graph import StateGraph
from langgraph.types import interrupt, Command
from langgraph.graph.message import add_messages

from typing import Annotated, Dict, List, Optional, TypedDict
import operator

##################################################################
## Define the authoritative state system (environment) for your use-case

class State(TypedDict):
    """The Graph State for your Agent System"""
    messages: Annotated[list, add_messages] = []
    agent_dict: Dict[str, dict]
    speakers: List[str] = []  ## <- use this to keep track of routing/enqueueing

##################################################################
## Define the operations (Nodes) that can happen on your environment

def agent(state: State):
    """Edge option where transition is generated at runtime"""
    agent_dict = state.get("agent_dict")
    current_speaker = state.get("speakers")[-1]
    ## TODO: If a speaker is retrieved properly, construct the agent connector,
    ## generate the response, and route to the appropriate next speaker.
    if current_speaker in agent_dict:
        current_agent = Agent(**agent_dict[current_speaker])
        response = current_agent(**state)[0]
        return Command(update={
            "messages": [("ai", str(response))], 
            "speakers": [response.get("route")],
        }, goto="agent")

##################################################################
## Define the system that organizes your nodes (and maybe edges)

builder = StateGraph(State)
builder.add_node("agent", agent)
builder.add_edge(START, "agent")  ## A start node is always necessary

<hr><br>

### **第五部分:** 反思这个练习

这可能是您今天实现的最难的系统了。我们必须遵循 LangGraph 的逻辑，定义一些自定义的、非直观的工具，并在每一步中验证决策。到现在为止，您可能已经意识到，这比我们在 CrewAI 中的方法要**困难得多**，这没问题！LangGraph 的吸引力部分就在于它实际上是一个高度可定制的解决方案，能够相对轻松地扩展到生产环境，并且在任何级别上都带有高度的可观察性和控制力。

您可能还记得，LangGraph 的 ReAct 循环在我们模型中并不能很好地直接使用，尽管它是以合理的方式实现的，因此不同的模型实际上会表现得更好。我们完全可以将这一抽象压缩，按照我们想要的方式进行调整，正是这个特点值得我们欣赏。而且，我们还需要提供一些实践机会作为评估前的准备，所以... 一切都是有意义的！

**在下一部分，请准备好尝试评估，看您是否能基于我们今天讨论的工具构建一个有趣的研究智能体！（但在此之前，热身活动可能会很有趣）**